# Sentiment Analysis
Author Brian Tam, 10/16/2020

Addition NLP work that could be tacked onto the topic features for the modeling process. These include:

10. Incoperated Vader sentiment Analysis into my model
11. Parts of speech counting into my model

In [9]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import numpy as np

# Import spacy to do NLP
import spacy

# import VADER for sentiment analysis 
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

from sklearn.feature_extraction.text import CountVectorizer

In [10]:
# Postgres info to connect
connection_args = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'dbname': 'myers_briggs',    # DB that we are connecting to
}

connection = pg.connect(**connection_args)  # What is that "**" there??

query = "SELECT * FROM cleaned_posts;"

df = pd.read_sql(query, connection)
df

,type,E/I,N/S,F/T,P/J,post,post_no_links,spacy_post,clean_post
0,INFJ,0,1,1,0,"""['http://www.youtube.com/watch?v=qsXHcwe3krw'...",What has been the most life-changing experienc...,life change experience life perc experience im...,life change experience life perc experience im...
1,ENTP,1,1,0,1,"""[""""I'm finding the lack of me in these posts ...",I'm finding the lack of me in these posts very...,find lack post alarming sex boring position ex...,find lack post alarming sex boring position ex...
2,INTP,0,1,0,1,"""['Good one _____ https://www.youtube.com/w...","Of course, to which I say I know; that's my bl...",course know blessing curse absolutely positive...,course know blessing curse absolutely positive...
3,INTJ,0,1,0,0,"""['Dear INTP, I enjoyed our conversation the...","Dear INTP, I enjoyed our conversation the ot...",dear intp enjoy conversation day esoteric gabb...,dear intp enjoy conversation day esoteric gabb...
4,ENTJ,1,1,0,0,"""[""""You're fired."""", """"That's another silly mi...",You're fired. That's another silly misconcepti...,fire silly misconception approach logically ke...,fire silly misconception approach logically ke...
...,...,...,...,...,...,...,...,...,...
8670,ENTP,1,1,0,1,"""['I think generally people experience post tr...",I think generally people experience post traum...,think generally people experience post trauma ...,think generally people experience post trauma ...
8671,INTJ,0,1,0,0,"""[""""Here's a planned stress relieving activity...",Here's a planned stress relieving activity tha...,planned stress relieve activity work ... day g...,planned stress relieve activity work day guy ...
8672,INFJ,0,1,1,0,"""[""""I'm not sure about a method for picking ou...",I'm not sure about a method for picking out IN...,sure method pick infj musical artist throw cau...,sure method pick infj musical artist throw cau...
8673,ISFP,0,0,1,1,"""['https://www.youtube.com/watch?v=t8edHB_h908...",Especially on websites that have become a have...,especially website haven neo nazis perc. nerd ...,especially website haven neo nazis perc nerd l...


In [4]:
df = df.dropna()

# VADOR Sentiment analysis

In [25]:
example = df.post_no_links.iloc[0:3].apply(lambda x: sid.polarity_scores(x))

In [26]:
#load VADER
analyzer = SentimentIntensityAnalyzer()
#Add VADER metrics to dataframe
df['compound'] = [analyzer.polarity_scores(v)['compound'] for v in df['post_no_links']]
df['neg'] = [analyzer.polarity_scores(v)['neg'] for v in df['post_no_links']]
df['neu'] = [analyzer.polarity_scores(v)['neu'] for v in df['post_no_links']]
df['pos'] = [analyzer.polarity_scores(v)['pos'] for v in df['post_no_links']]

<ipython-input-26-ebd6ee2825eb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['compound'] = [analyzer.polarity_scores(v)['compound'] for v in df['post_no_links']]
<ipython-input-26-ebd6ee2825eb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['neg'] = [analyzer.polarity_scores(v)['neg'] for v in df['post_no_links']]
<ipython-input-26-ebd6ee2825eb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [22]:
example[0]

{'neg': 0.084, 'neu': 0.778, 'pos': 0.138, 'compound': 0.9789}

In [33]:
sentiment_df = df.iloc[:,-4:]

In [34]:
import d6tstack

# fast CSV to SQL import - see SQL examples notebook
d6tstack.utils.pd_to_psql(sentiment_df, 
                          'postgresql+psycopg2://briantam: @localhost/myers_briggs', 
                          'polarity_scores', 
                          if_exists='replace',
                          sep='\t')


True

In [37]:
sid.polarity_scores()

{'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'compound': 0.6369}

In [ ]:
# Parts of

In [38]:
df['POS'] = df.post_no_links.apply(lambda x:[token.pos_ for token in nlp(x)]) 

<ipython-input-38-87b6ac1237c0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['POS'] = df.post_no_links.apply(lambda x:[token.pos_ for token in nlp(x)])


In [58]:
df.POS[0].unique

AttributeError: 'list' object has no attribute 'unique'

In [51]:
POS_df = pd.get_dummies(df.POS.apply(pd.Series).stack()).sum(level=0)

In [59]:
POS_df

,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SPACE,SYM,VERB,X
0,45.0,55.0,37.0,30.0,12.0,65.0,5.0,105.0,6.0,17.0,44.0,16.0,80.0,16.0,6.0,1.0,70.0,3.0
1,91.0,91.0,75.0,97.0,44.0,117.0,10.0,174.0,15.0,40.0,174.0,54.0,165.0,40.0,20.0,5.0,170.0,5.0
2,81.0,66.0,95.0,66.0,26.0,80.0,13.0,141.0,18.0,32.0,96.0,18.0,146.0,23.0,18.0,3.0,123.0,0.0
3,81.0,96.0,97.0,105.0,35.0,125.0,9.0,166.0,21.0,38.0,152.0,38.0,181.0,37.0,28.0,4.0,151.0,1.0
4,88.0,84.0,68.0,113.0,34.0,117.0,9.0,172.0,5.0,42.0,122.0,28.0,145.0,14.0,13.0,2.0,135.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8670,126.0,114.0,144.0,109.0,62.0,191.0,11.0,233.0,13.0,50.0,160.0,29.0,170.0,40.0,15.0,1.0,222.0,1.0
8671,70.0,89.0,102.0,91.0,49.0,118.0,6.0,170.0,8.0,46.0,154.0,21.0,138.0,35.0,9.0,2.0,183.0,0.0
8672,76.0,117.0,92.0,101.0,28.0,126.0,12.0,182.0,15.0,49.0,125.0,70.0,230.0,29.0,54.0,3.0,156.0,1.0
8673,72.0,61.0,65.0,85.0,28.0,83.0,5.0,126.0,7.0,33.0,105.0,36.0,106.0,36.0,13.0,2.0,112.0,0.0


In [65]:
df.iloc[:,-5:-1]

,compound,neg,neu,pos
0,0.9789,0.084,0.778,0.138
1,0.9993,0.086,0.717,0.196
2,0.9990,0.081,0.718,0.201
3,0.9982,0.056,0.811,0.132
4,0.9727,0.124,0.731,0.146
...,...,...,...,...
8670,0.9993,0.079,0.753,0.168
8671,0.9991,0.095,0.719,0.186
8672,0.9988,0.065,0.789,0.146
8673,0.9959,0.105,0.730,0.165


In [52]:
df[df['N/S']==0]

,type,E/I,N/S,F/T,P/J,post,post_no_links,spacy_post,clean_post
31,ISFP,0,0,1,1,"""['They paint without numbers', """"I'd guess at...",They paint without numbers I'd guess at istp t...,paint number guess istp good bud esfp funny en...,paint number guess istp good bud esfp funny en...
40,ISTP,0,0,0,1,"""[""""I got 593. From what I've read about the ...",I got 593. From what I've read about the Enne...,593 read enneagram 953 read lot 9 mistype 5 gt...,read enneagram read lot mistype gtfo feele...
48,ISFJ,0,0,1,0,"""['I love feeling affectionate for the one I l...",I love feeling affectionate for the one I love...,love feeling affectionate love care care deepl...,love feeling affectionate love care care deepl...
52,ISFJ,0,0,1,0,"""[""""I'm currently rooting around in the forum,...","I'm currently rooting around in the forum, loo...",currently root forum look place rant think thr...,currently root forum look place rant think thr...
66,ISFP,0,0,1,1,"""[""""True, sadly not many have felt like partic...","True, sadly not many have felt like participat...",true sadly feel like participate unsure helpfu...,true sadly feel like participate unsure helpfu...
...,...,...,...,...,...,...,...,...,...
8654,ISTP,0,0,0,1,"""['-Being alone -Being insensitive -Over thin...",-Being alone -Being insensitive -Over thinkin...,-bee -being insensitive -over think loud socia...,bee being insensitive over think loud social s...
8655,ISFJ,0,0,1,0,"""['Being assertive and standing up for myself ...",Being assertive and standing up for myself is ...,assertive stand struggle day worrying think ne...,assertive stand struggle day worrying think ne...
8657,ISTJ,0,0,0,0,"""[""""Assuming I had a hand in it, I always at l...","Assuming I had a hand in it, I always at least...",assume hand consider possibility culpability l...,assume hand consider possibility culpability l...
8661,ESFJ,1,0,1,0,"""[""""Don't worry, I do that too when a car come...","Don't worry, I do that too when a car comes cl...",worry car come close change lane outrun assume...,worry car come close change lane outrun assume...


# EXPORT to SQL tables

In [66]:
import d6tstack

# fast CSV to SQL import - see SQL examples notebook
d6tstack.utils.pd_to_psql(df.iloc[:,-5:-1], 
                          'postgresql+psycopg2://briantam: @localhost/myers_briggs', 
                          'sentiment', 
                          if_exists='replace',
                          sep='\t')

True

In [61]:
# fast CSV to SQL import - see SQL examples notebook
d6tstack.utils.pd_to_psql(POS_df, 
                          'postgresql+psycopg2://briantam: @localhost/myers_briggs', 
                          'POS', 
                          if_exists='replace',
                          sep='\t')

True

In [6]:
# fast CSV to SQL import - see SQL examples notebook
d6tstack.utils.pd_to_psql(POS_df, 
                          'postgresql+psycopg2://briantam: @localhost/myers_briggs', 
                          'clean_posts', 
                          if_exists='replace',
                          sep='\t')

NameError: name 'd6tstack' is not defined

# Appendix

In [7]:
import spacy
nlp = spacy.load('en_core_web_sm')

# Import spaCy Matcher
from spacy.matcher import Matcher

# Initialize the matcher with the spaCy vocabulary
matcher = Matcher(nlp.vocab)

doc = nlp("Some people start their day with lemon water")

# Define rule
pattern = [{'TEXT': 'lemon'}, {'TEXT': 'water'}]

# Add rule
matcher.add('rule_1', None, pattern)

In [2]:
matches = matcher(doc)
matches

[(7604275899133490726, 6, 8)]

In [3]:

doc1 = nlp("You read this book")
doc2 = nlp("I will book my ticket")

pattern = [{'TEXT': 'book', 'POS': 'NOUN'}]

# Initialize the matcher with the shared vocab
matcher = Matcher(nlp.vocab)
matcher.add('rule_2', None, pattern)

In [4]:
matches = matcher(doc1)
matches

[(375134486054924901, 3, 4)]